In [1]:
import numpy as np
import scipy.io
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

# load `.mat` file
data = scipy.io.loadmat('face.mat')

# Images
# N: number of images
# D: number of pixels
X = data['X']  # shape: [D x N]
y = data['l']  # shape: [1 x N]

assert(X.shape[1] == y.shape[1])
# Number of images
D, N = X.shape

# Fix the random seed
np.random.seed(13)

# Cardinality of labels
_card = len(set(y.ravel()))

# Step splitting of dataset
_step = int(N / _card)

# Shape boundaries
_bounds = np.arange(0, N+1, _step)

# Shapes
shapes = list(zip(_bounds[:-1], _bounds[1:]))

# Training Mask
_mask = []

for _shape in shapes:
    _idx = np.random.choice(
        np.arange(*_shape), int(0.8 * _step), replace=False)
    _mask.append(_idx)

mask_train = np.array(_mask).ravel()

mask_test = np.array(list(set(np.arange(0, N)) - set(mask_train)))

# Partition dataset to train and test sets
X_train, X_test = X[:, mask_train], X[:, mask_test]
y_train, y_test = y[:, mask_train], y[:, mask_test]

In [2]:
class PCA(object):
    """Principle Component Analysis."""

    def __init__(self, n_comps=5, standard=True):
        """Contructor.
        Parameters
        ----------
        n_comps: int
            Number of principle components
        """
        self._fitted = False
        self.n_comps = n_comps
        self.standard = standard
        self.mean = None
        self.U = None

    def fit(self, X):
        """Fit PCA according to `X.cov()`.
        Parameters
        ----------
        X: numpy.ndarray
            Features matrix
        Returns
        -------
        array: numpy.ndarray
            Transformed features matrix
        """
        self.D, N = X.shape
        self.mean = X.mean(axis=1).reshape(-1, 1)
        # center data
        A = X - self.mean
        # covariance matrix
        S = (1 / N) * np.dot(A.T, A)
        
        _l, _v = np.linalg.eig(S)

        _indexes = np.argsort(_l)[::-1]

        # Sorted eigenvalues and eigenvectors
        l, v = _l[_indexes], _v[:, _indexes]

        V = v[:, :self.n_comps]

        _U = np.dot(A, V)

        self.U = _U / np.apply_along_axis(np.linalg.norm, 0, _U)

        W = np.dot(self.U.T, A)

        if self.standard:
            self.W_mean = np.mean(W, axis=1)
            self.W_std = np.std(W, axis=1)

        self._fitted = True

        if self.standard:
            return ((W.T - self.W_mean) / self.W_std).T
        else:
            return W

    def transform(self, X):
        """Transform `X` by projecting it to PCA feature space.
        Parameters
        ----------
        X: numpy.ndarray
            Features matrix
        Returns
        -------
        array: numpy.ndarray
            Transformed features matrix
        """

        Phi = X - self.mean

        W = np.dot(self.U.T, Phi)

        if self.standard:
            return ((W.T - self.W_mean) / self.W_std).T
        else:
            return W

    def reconstruct(self, W):
        """Recontruct compressed data.
        Parameters
        ----------
        W: numpy.ndarray
            Projection coefficients matrix
        Returns
        -------
        X_hat: numpy.ndarray
            Reconstructed features matrix
        """
        A_hat = np.dot(self.U, W).reshape(-1, 1)
        A_hat = A_hat + self.mean
        return A_hat

In [64]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# KNN Classifer
from sklearn.neighbors import KNeighborsClassifier

_card = 52
D, N = X_train.shape

M_pca = 1
M_lda = 1


standard = False

M__pca_ideal = None
M__lda_ideal = None
acc_max = 0

while M_pca <= (N-_card):
    M_lda = 1
    while M_lda <= (_card-1):

        pca = PCA(n_comps=M_pca, standard=standard)
        W_train = pca.fit(X_train)
        

        lda = LinearDiscriminantAnalysis(n_components=M_lda)
        W_train_2 = lda.fit_transform(W_train.T, y_train.T.ravel())

        nn = KNeighborsClassifier(n_neighbors=1)
        nn.fit(W_train_2, y_train.T.ravel())

        W_test = pca.transform(X_test)

        W_test_2 = lda.transform(W_test.T)

        acc = nn.score(W_test_2, y_test.T.ravel())

        print('M_pca = ', M_pca, ', M_lda = ', M_lda,' --->  Accuracy = %.2f%%' % (acc * 100))
        
        if (acc > acc_max):
            M__pca_ideal = M_pca
            M__lda_ideal = M_lda
            acc_max = acc

        M_lda = M_lda + 1
        
    M_pca = M_pca + 1
    
print ("Accuracy is maximum for M__pca = ", M__pca_ideal, ", M_lda = ", M__lda_ideal, " with accuracy of %.2f%%"% (acc_max * 100), ".")

#Ideal: M_pca =  147 , M_lda =  46  --->  Accuracy = 94.23%

/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


M_pca =  1 , M_lda =  1  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  2  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  3  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  4  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  5  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  6  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  7  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  8  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  9  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  10  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  11  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  12  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  13  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  14  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  15  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  16  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  17  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  18  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  19  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  20  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  21  ---

M_pca =  4 , M_lda =  15  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  16  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  17  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  18  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  19  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  20  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  21  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  22  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  23  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  24  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  25  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  26  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  27  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  28  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  29  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  30  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  31  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  32  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  33  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  34  --->  Accuracy = 25.00%


M_pca =  7 , M_lda =  27  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  28  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  29  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  30  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  31  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  32  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  33  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  34  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  35  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  36  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  37  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  38  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  39  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  40  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  41  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  42  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  43  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  44  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  45  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  46  --->  Accuracy = 37.50%


M_pca =  10 , M_lda =  38  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  39  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  40  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  41  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  42  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  43  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  44  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  45  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  46  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  47  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  48  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  49  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  50  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  51  --->  Accuracy = 52.88%
M_pca =  11 , M_lda =  1  --->  Accuracy = 5.77%
M_pca =  11 , M_lda =  2  --->  Accuracy = 15.38%
M_pca =  11 , M_lda =  3  --->  Accuracy = 28.85%
M_pca =  11 , M_lda =  4  --->  Accuracy = 35.58%
M_pca =  11 , M_lda =  5  --->  Accuracy = 37.50%
M_pca =  11 , M_lda =  6  --->  Accur

...

M_pca =  357 , M_lda =  17  --->  Accuracy = 45.19%
M_pca =  357 , M_lda =  18  --->  Accuracy = 45.19%
M_pca =  357 , M_lda =  19  --->  Accuracy = 47.12%
M_pca =  357 , M_lda =  20  --->  Accuracy = 47.12%
M_pca =  357 , M_lda =  21  --->  Accuracy = 47.12%
M_pca =  357 , M_lda =  22  --->  Accuracy = 47.12%
M_pca =  357 , M_lda =  23  --->  Accuracy = 47.12%
M_pca =  357 , M_lda =  24  --->  Accuracy = 47.12%
M_pca =  357 , M_lda =  25  --->  Accuracy = 47.12%
M_pca =  357 , M_lda =  26  --->  Accuracy = 47.12%
M_pca =  357 , M_lda =  27  --->  Accuracy = 47.12%
M_pca =  357 , M_lda =  28  --->  Accuracy = 48.08%
M_pca =  357 , M_lda =  29  --->  Accuracy = 47.12%
M_pca =  357 , M_lda =  30  --->  Accuracy = 47.12%
M_pca =  357 , M_lda =  31  --->  Accuracy = 47.12%
M_pca =  357 , M_lda =  32  --->  Accuracy = 47.12%
M_pca =  357 , M_lda =  33  --->  Accuracy = 47.12%
M_pca =  357 , M_lda =  34  --->  Accuracy = 48.08%
M_pca =  357 , M_lda =  35  --->  Accuracy = 48.08%
M_pca =  357

M_pca =  360 , M_lda =  24  --->  Accuracy = 36.54%
M_pca =  360 , M_lda =  25  --->  Accuracy = 36.54%
M_pca =  360 , M_lda =  26  --->  Accuracy = 37.50%
M_pca =  360 , M_lda =  27  --->  Accuracy = 37.50%
M_pca =  360 , M_lda =  28  --->  Accuracy = 38.46%
M_pca =  360 , M_lda =  29  --->  Accuracy = 38.46%
M_pca =  360 , M_lda =  30  --->  Accuracy = 39.42%
M_pca =  360 , M_lda =  31  --->  Accuracy = 39.42%
M_pca =  360 , M_lda =  32  --->  Accuracy = 38.46%
M_pca =  360 , M_lda =  33  --->  Accuracy = 39.42%
M_pca =  360 , M_lda =  34  --->  Accuracy = 39.42%
M_pca =  360 , M_lda =  35  --->  Accuracy = 40.38%
M_pca =  360 , M_lda =  36  --->  Accuracy = 40.38%
M_pca =  360 , M_lda =  37  --->  Accuracy = 40.38%
M_pca =  360 , M_lda =  38  --->  Accuracy = 40.38%
M_pca =  360 , M_lda =  39  --->  Accuracy = 40.38%
M_pca =  360 , M_lda =  40  --->  Accuracy = 40.38%
M_pca =  360 , M_lda =  41  --->  Accuracy = 40.38%
M_pca =  360 , M_lda =  42  --->  Accuracy = 40.38%
M_pca =  360

M_pca =  363 , M_lda =  31  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  32  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  33  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  34  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  35  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  36  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  37  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  38  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  39  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  40  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  41  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  42  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  43  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  44  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  45  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  46  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  47  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  48  --->  Accuracy = 22.12%
M_pca =  363 , M_lda =  49  --->  Accuracy = 22.12%
M_pca =  363

/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


M_pca =  364 , M_lda =  2  --->  Accuracy = 7.69%
M_pca =  364 , M_lda =  3  --->  Accuracy = 7.69%
M_pca =  364 , M_lda =  4  --->  Accuracy = 11.54%
M_pca =  364 , M_lda =  5  --->  Accuracy = 12.50%
M_pca =  364 , M_lda =  6  --->  Accuracy = 12.50%
M_pca =  364 , M_lda =  7  --->  Accuracy = 14.42%
M_pca =  364 , M_lda =  8  --->  Accuracy = 18.27%
M_pca =  364 , M_lda =  9  --->  Accuracy = 21.15%
M_pca =  364 , M_lda =  10  --->  Accuracy = 22.12%
M_pca =  364 , M_lda =  11  --->  Accuracy = 23.08%
M_pca =  364 , M_lda =  12  --->  Accuracy = 23.08%
M_pca =  364 , M_lda =  13  --->  Accuracy = 24.04%
M_pca =  364 , M_lda =  14  --->  Accuracy = 24.04%
M_pca =  364 , M_lda =  15  --->  Accuracy = 24.04%
M_pca =  364 , M_lda =  16  --->  Accuracy = 24.04%
M_pca =  364 , M_lda =  17  --->  Accuracy = 24.04%
M_pca =  364 , M_lda =  18  --->  Accuracy = 25.00%
M_pca =  364 , M_lda =  19  --->  Accuracy = 25.96%
M_pca =  364 , M_lda =  20  --->  Accuracy = 25.96%
M_pca =  364 , M_lda =

In [20]:
# LDA-PCA using scikit learn PCA function

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

# KNN Classifer
from sklearn.neighbors import KNeighborsClassifier

_card = 52
D, N = X_train.shape
        
    
M_pca = 1
M_lda = 1


standard = False

M__pca_ideal = None
M__lda_ideal = None
acc_max = 0

while M_pca <= (N-_card):
    M_lda = 1
    while M_lda <= (_card-1):

        pca = PCA(n_components=M_pca)
        W_train = pca.fit_transform(X_train.T)

        lda = LinearDiscriminantAnalysis(n_components=M_lda)
        W_train_2 = lda.fit_transform(W_train, y_train.T.ravel())

        nn = KNeighborsClassifier(n_neighbors=1)
        nn.fit(W_train_2, y_train.T.ravel())

        W_test = pca.transform(X_test.T)

        W_test_2 = lda.transform(W_test)

        acc = nn.score(W_test_2, y_test.T.ravel())

        print('M_pca = ', M_pca, ', M_lda = ', M_lda,' --->  Accuracy = %.2f%%' % (acc * 100))
        
        if (acc > acc_max):
            M__pca_ideal = M_pca
            M__lda_ideal = M_lda
            acc_max = acc

        M_lda = M_lda + 1
        
    M_pca = M_pca + 1
    
print ("Accuracy is maximum for M__pca = ", M__pca_ideal, ", M_lda = ", M__lda_ideal, " with accuracy of %.2f%%"% (acc_max * 100), ".")

#Accuracy is maximum for M__pca =  150 , M_lda =  47  with accuracy of 94.23% .

/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


M_pca =  1 , M_lda =  1  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  2  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  3  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  4  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  5  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  6  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  7  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  8  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  9  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  10  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  11  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  12  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  13  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  14  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  15  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  16  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  17  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  18  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  19  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  20  --->  Accuracy = 4.81%
M_pca =  1 , M_lda =  21  ---

M_pca =  4 , M_lda =  15  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  16  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  17  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  18  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  19  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  20  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  21  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  22  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  23  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  24  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  25  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  26  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  27  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  28  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  29  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  30  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  31  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  32  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  33  --->  Accuracy = 25.00%
M_pca =  4 , M_lda =  34  --->  Accuracy = 25.00%


M_pca =  7 , M_lda =  30  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  31  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  32  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  33  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  34  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  35  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  36  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  37  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  38  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  39  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  40  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  41  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  42  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  43  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  44  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  45  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  46  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  47  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  48  --->  Accuracy = 37.50%
M_pca =  7 , M_lda =  49  --->  Accuracy = 37.50%


M_pca =  10 , M_lda =  43  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  44  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  45  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  46  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  47  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  48  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  49  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  50  --->  Accuracy = 52.88%
M_pca =  10 , M_lda =  51  --->  Accuracy = 52.88%
M_pca =  11 , M_lda =  1  --->  Accuracy = 5.77%
M_pca =  11 , M_lda =  2  --->  Accuracy = 15.38%
M_pca =  11 , M_lda =  3  --->  Accuracy = 28.85%
M_pca =  11 , M_lda =  4  --->  Accuracy = 35.58%
M_pca =  11 , M_lda =  5  --->  Accuracy = 37.50%
M_pca =  11 , M_lda =  6  --->  Accuracy = 39.42%
M_pca =  11 , M_lda =  7  --->  Accuracy = 47.12%
M_pca =  11 , M_lda =  8  --->  Accuracy = 51.92%
M_pca =  11 , M_lda =  9  --->  Accuracy = 50.96%
M_pca =  11 , M_lda =  10  --->  Accuracy = 51.92%
M_pca =  11 , M_lda =  11  --->  Accuracy

...

M_pca =  297 , M_lda =  15  --->  Accuracy = 74.04%
M_pca =  297 , M_lda =  16  --->  Accuracy = 73.08%
M_pca =  297 , M_lda =  17  --->  Accuracy = 72.12%
M_pca =  297 , M_lda =  18  --->  Accuracy = 75.00%
M_pca =  297 , M_lda =  19  --->  Accuracy = 72.12%
M_pca =  297 , M_lda =  20  --->  Accuracy = 76.92%
M_pca =  297 , M_lda =  21  --->  Accuracy = 73.08%
M_pca =  297 , M_lda =  22  --->  Accuracy = 75.96%
M_pca =  297 , M_lda =  23  --->  Accuracy = 75.00%
M_pca =  297 , M_lda =  24  --->  Accuracy = 79.81%
M_pca =  297 , M_lda =  25  --->  Accuracy = 77.88%
M_pca =  297 , M_lda =  26  --->  Accuracy = 78.85%
M_pca =  297 , M_lda =  27  --->  Accuracy = 78.85%
M_pca =  297 , M_lda =  28  --->  Accuracy = 79.81%
M_pca =  297 , M_lda =  29  --->  Accuracy = 79.81%
M_pca =  297 , M_lda =  30  --->  Accuracy = 80.77%
M_pca =  297 , M_lda =  31  --->  Accuracy = 76.92%
M_pca =  297 , M_lda =  32  --->  Accuracy = 83.65%
M_pca =  297 , M_lda =  33  --->  Accuracy = 77.88%
M_pca =  297

M_pca =  300 , M_lda =  21  --->  Accuracy = 75.00%
M_pca =  300 , M_lda =  22  --->  Accuracy = 75.96%
M_pca =  300 , M_lda =  23  --->  Accuracy = 75.96%
M_pca =  300 , M_lda =  24  --->  Accuracy = 78.85%
M_pca =  300 , M_lda =  25  --->  Accuracy = 80.77%
M_pca =  300 , M_lda =  26  --->  Accuracy = 75.00%
M_pca =  300 , M_lda =  27  --->  Accuracy = 79.81%
M_pca =  300 , M_lda =  28  --->  Accuracy = 78.85%
M_pca =  300 , M_lda =  29  --->  Accuracy = 76.92%
M_pca =  300 , M_lda =  30  --->  Accuracy = 78.85%
M_pca =  300 , M_lda =  31  --->  Accuracy = 80.77%
M_pca =  300 , M_lda =  32  --->  Accuracy = 80.77%
M_pca =  300 , M_lda =  33  --->  Accuracy = 76.92%
M_pca =  300 , M_lda =  34  --->  Accuracy = 84.62%
M_pca =  300 , M_lda =  35  --->  Accuracy = 78.85%
M_pca =  300 , M_lda =  36  --->  Accuracy = 79.81%
M_pca =  300 , M_lda =  37  --->  Accuracy = 77.88%
M_pca =  300 , M_lda =  38  --->  Accuracy = 80.77%
M_pca =  300 , M_lda =  39  --->  Accuracy = 81.73%
M_pca =  300

M_pca =  303 , M_lda =  27  --->  Accuracy = 77.88%
M_pca =  303 , M_lda =  28  --->  Accuracy = 80.77%
M_pca =  303 , M_lda =  29  --->  Accuracy = 79.81%
M_pca =  303 , M_lda =  30  --->  Accuracy = 78.85%
M_pca =  303 , M_lda =  31  --->  Accuracy = 75.00%
M_pca =  303 , M_lda =  32  --->  Accuracy = 78.85%
M_pca =  303 , M_lda =  33  --->  Accuracy = 79.81%
M_pca =  303 , M_lda =  34  --->  Accuracy = 77.88%
M_pca =  303 , M_lda =  35  --->  Accuracy = 79.81%
M_pca =  303 , M_lda =  36  --->  Accuracy = 80.77%
M_pca =  303 , M_lda =  37  --->  Accuracy = 77.88%
M_pca =  303 , M_lda =  38  --->  Accuracy = 77.88%
M_pca =  303 , M_lda =  39  --->  Accuracy = 75.96%
M_pca =  303 , M_lda =  40  --->  Accuracy = 82.69%
M_pca =  303 , M_lda =  41  --->  Accuracy = 77.88%
M_pca =  303 , M_lda =  42  --->  Accuracy = 79.81%
M_pca =  303 , M_lda =  43  --->  Accuracy = 79.81%
M_pca =  303 , M_lda =  44  --->  Accuracy = 84.62%
M_pca =  303 , M_lda =  45  --->  Accuracy = 80.77%
M_pca =  303

M_pca =  306 , M_lda =  33  --->  Accuracy = 78.85%
M_pca =  306 , M_lda =  34  --->  Accuracy = 76.92%
M_pca =  306 , M_lda =  35  --->  Accuracy = 83.65%
M_pca =  306 , M_lda =  36  --->  Accuracy = 77.88%
M_pca =  306 , M_lda =  37  --->  Accuracy = 77.88%
M_pca =  306 , M_lda =  38  --->  Accuracy = 77.88%
M_pca =  306 , M_lda =  39  --->  Accuracy = 80.77%
M_pca =  306 , M_lda =  40  --->  Accuracy = 80.77%
M_pca =  306 , M_lda =  41  --->  Accuracy = 83.65%
M_pca =  306 , M_lda =  42  --->  Accuracy = 80.77%
M_pca =  306 , M_lda =  43  --->  Accuracy = 78.85%
M_pca =  306 , M_lda =  44  --->  Accuracy = 80.77%
M_pca =  306 , M_lda =  45  --->  Accuracy = 80.77%
M_pca =  306 , M_lda =  46  --->  Accuracy = 79.81%
M_pca =  306 , M_lda =  47  --->  Accuracy = 81.73%
M_pca =  306 , M_lda =  48  --->  Accuracy = 80.77%
M_pca =  306 , M_lda =  49  --->  Accuracy = 78.85%
M_pca =  306 , M_lda =  50  --->  Accuracy = 79.81%
M_pca =  306 , M_lda =  51  --->  Accuracy = 80.77%
M_pca =  307

KeyboardInterrupt: 

In [127]:


from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

from sklearn import model_selection
from sklearn.ensemble import VotingClassifier

D, N = X_train.shape

standard = False
#M__pca_ideal = 147
#M__lda_ideal = 46

print ('M__pca_ideal = ', M__pca_ideal)
print ('M__lda_ideal = ', M__lda_ideal)

M_pca_bag = N-1

M_pca = 150 #M__pca_ideal
M_lda = 47 #M__lda_ideal

n_est = 10

estimators = [('pca', PCA(n_components=M_pca)), ('lda', LinearDiscriminantAnalysis(n_components=M_lda)), ('knn', KNeighborsClassifier(n_neighbors=1))]

base_est = Pipeline (estimators)

print (X_train.shape)
print (y_train.shape)

base_est.fit(X_train.T, y_train.T.ravel())

acc = base_est.score(X_test.T, y_test.T.ravel())
print ('Accuracy of base estimator with no pre PCA = %.2f%%' % (acc * 100))


pca = PCA(n_components=M_pca_bag)
W_train = pca.fit_transform(X_train.T)
W_test = pca.transform(X_test.T)

base_est.fit(W_train, y_train.T.ravel())

acc = base_est.score(W_test, y_test.T.ravel())
print ('Accuracy of base estimator with pre PCA applied = %.2f%%' % (acc * 100))

bagging = BaggingClassifier(base_estimator=base_est,
                            max_samples=1.0,
                            max_features=1.0,
                            bootstrap=True,
                            #bootstrap_features=True,
                            n_estimators=n_est)

print (W_train.shape)

bagging = bagging.fit(W_train, y_train.T.ravel())

acc = bagging.score(W_test, y_test.T.ravel())
print ('Accuracy of ensemble estimator = %.2f%%' % (acc * 100))



seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
results = model_selection.cross_val_score(bagging, W_train, y_train.T.ravel(), cv=kfold)
print('Cross validation mean accuracy = %.2f%%' % (results.mean() * 100))

sub_model_accuracies = []

sub_estimators = []

for i, estimator in enumerate(bagging.estimators_):
    print(estimator)
    sub_model_acc = estimator.score(W_test, y_test.T.ravel())
    print ('Accuracy of sub model ', i+1, ' = %.2f%%' % (sub_model_acc * 100))
    sub_model_accuracies.append(sub_model_acc)
    name = 'est'+str(i+1)
    sub_estimators.append((name, estimator))
    
ave_sub_model_acc = sum(sub_model_accuracies)/n_est

print ('Average accuracy of sub models = %.2f%%' % (ave_sub_model_acc * 100))


voting = VotingClassifier(estimators=sub_estimators, voting='soft')
voting = voting.fit(W_train, y_train.T.ravel())
acc = voting.score(W_test, y_test.T.ravel())
print ('Accuracy of voting = %.2f%%' % (acc * 100))


M__pca_ideal =  150
M__lda_ideal =  47
(2576, 416)
(1, 416)
Accuracy of base estimator with no pre PCA = 90.38%


/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


Accuracy of base estimator with pre PCA applied = 91.35%
(416, 415)
Accuracy of ensemble estimator = 84.62%


/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. 

Cross validation mean accuracy = 6.51%
Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=150,
  random_state=1704556621, svd_solver='auto', tol=0.0, whiten=False)), ('lda', LinearDiscriminantAnalysis(n_components=47, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)), ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform'))])
Accuracy of sub model  1  = 0.00%
Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=150,
  random_state=908355190, svd_solver='auto', tol=0.0, whiten=False)), ('lda', LinearDiscriminantAnalysis(n_components=47, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)), ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1,

/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. 

Accuracy of voting = 91.35%


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [192]:


from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

from sklearn import model_selection
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.base import clone

D, N = X_train.shape

standard = False
#M__pca_ideal = 147
#M__lda_ideal = 46

print ('M__pca_ideal = ', M__pca_ideal)
print ('M__lda_ideal = ', M__lda_ideal)

M_pca_bag = N-1

M_pca = 150 #M__pca_ideal
M_lda = 47 #M__lda_ideal

n_estimators = 10
max_samples = 0.8
max_features=1.0

estimators = [('pca', PCA(n_components=M_pca)), ('lda', LinearDiscriminantAnalysis(n_components=M_lda)), ('knn', KNeighborsClassifier(n_neighbors=1))]

base_est = Pipeline (estimators)

base_est.fit(X_train.T, y_train.T.ravel())

acc = base_est.score(X_test.T, y_test.T.ravel())
print ('Accuracy of base estimator with no pre PCA = %.2f%%' % (acc * 100))


pca = PCA(n_components=M_pca_bag)
W_train = pca.fit_transform(X_train.T)
W_test = pca.transform(X_test.T)

base_est.fit(W_train, y_train.T.ravel())

acc = base_est.score(W_test, y_test.T.ravel())
print ('Accuracy of base estimator with pre PCA applied = %.2f%%' % (acc * 100))

estimators = []

for i in range (n_estimators):

    mask = np.random.choice(np.arange(N), int(max_samples * N), replace=False)

    mask = np.array(mask).ravel()

    W_bag = W_train[mask, :]
    y_bag = y_train[:, mask]
    
    estimator = clone(base_est)

    estimator.fit(W_bag, y_bag.T.ravel())
    
    name = 'est_'+str(i+1)
    estimators.append((name, estimator))
    
    sub_model_acc = estimator.score(W_test, y_test.T.ravel())
    print ('Accuracy of sub model ', i+1, ' = %.2f%%' % (sub_model_acc * 100))

y_hat = []

for w in W_test:
    prediction_sum = 0
    predictions = np.empty(n_estimators, dtype = np.int64)
    for i, (name, estimator) in enumerate(estimators):
        y = estimator.predict(w.reshape(1, -1))
        
        prediction_sum = prediction_sum + float(y[0])
        predictions[i] = int(y[0])
    prediction = round(prediction_sum/n_estimators)
    
    counts = np.bincount(predictions)
    y_hat.append(prediction)
    #y_hat.append(np.argmax(counts))
    
acc = accuracy_score(y_test.T, y_hat)
print ('Accuracy of ensemble estimator = %.2f%%' % (acc * 100))
    
        
    



M__pca_ideal =  150
M__lda_ideal =  47
Accuracy of base estimator with no pre PCA = 89.42%


/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


Accuracy of base estimator with pre PCA applied = 91.35%
Accuracy of sub model  1  = 81.73%
Accuracy of sub model  2  = 83.65%
Accuracy of sub model  3  = 78.85%
Accuracy of sub model  4  = 80.77%
Accuracy of sub model  5  = 78.85%


/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


Accuracy of sub model  6  = 83.65%
Accuracy of sub model  7  = 80.77%
Accuracy of sub model  8  = 84.62%
Accuracy of sub model  9  = 80.77%
Accuracy of sub model  10  = 83.65%
Accuracy of ensemble estimator = 54.81%
